<a href="https://colab.research.google.com/github/AleOi2/PSI3472/blob/master/Aula11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#resnet_transf.py - testado em Colab usando TF2
#Faz transfer learning usando ResNet50.
#No fim, faz aprendizagem descongelando todas camadas. Usando exemplo de
#https://medium.com/abraia/first-steps-with-transfer-learning-for-custom-image-classification-with-kerasb941601fcad5
import numpy as np;
import keras
import keras.backend as K;
from keras import optimizers, callbacks, regularizers;
from keras.regularizers import l2;
from keras.models import Sequential, Model;
from keras.layers import Dropout, Conv2D, MaxPooling2D, Dense, Flatten;
from keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from keras.preprocessing import image
import os; os.environ['TF_CPP_MIN_LOG_LEVEL']='3'
from google.colab import drive
drive.mount('/content/drive')

def leCsv(nomeDir, nomeArq, nl=0, nc=0):
  st=os.path.join(nomeDir,nomeArq);
  arq=open(st,"rt"); lines=arq.readlines(); arq.close();
  n=len(lines)
  linhas_separadas=[]
  for linha in lines:
    linha=linha.strip('\n'); linha=linha.split(';');
    linhas_separadas.append(linha);
    ay=np.empty((n),dtype='float32');
    ax=np.empty((n,nl,nc,3),dtype='float32');
    for i in range(len(linhas_separadas)):
      if i%10==0: print("Lendo arquivo",i,linha[0])
      linha=linhas_separadas[i];
      img_path=os.path.join(nomeDir,linha[0])
      t = image.load_img(img_path, target_size=(nl,nc))
      x = image.img_to_array(t)
      x = np.expand_dims(x, axis=0)
      ax[i] = preprocess_input(x)
      ay[i] = np.float32(linha[1]); #0=m ou 1=f
  return ax, ay;
#<<<<<<<<<<<<<<<<<<<<<<<< main <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
nomeprog="resnet_transf6"
#Original: 280x200, redimensionado: 224x224
num_classes=2; nl=224; nc=224
diretorioBd="/content/drive/My Drive/Colab Notebooks/psi3472/Aula11/feiCorCrop"
ax, ay = leCsv(diretorioBd,"treino.csv", nl=nl, nc=nc); #200 imagens
qx, qy = leCsv(diretorioBd,"teste.csv", nl=nl, nc=nc); #100 imagens
vx, vy = leCsv(diretorioBd,"valida.csv", nl=nl, nc=nc); #100 imagens
ay = keras.utils.to_categorical(ay, num_classes)
qy = keras.utils.to_categorical(qy, num_classes)
vy = keras.utils.to_categorical(vy, num_classes)
input_shape = (nl,nc,3); batch_size = 10;

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
base_model.summary()
x = base_model.output
x = Flatten()(x)
x = Dense(100, activation="relu")(x)
predictions = Dense(num_classes, activation="softmax")(x)
model = Model(inputs=base_model.input, outputs=predictions)

#Nao permite treinar base_model. So as camadas densas sao treinadas:
for layer in base_model.layers: layer.trainable = False
#Treina com learning rate grande
otimizador=keras.optimizers.Adam(lr=1e-3)
model.compile(otimizador, loss='categorical_crossentropy', metrics =['accuracy'])
model.fit(ax, ay, batch_size=batch_size, epochs=5, verbose=2, validation_data=(vx,vy))
score = model.evaluate(ax, ay, verbose=0); print('Training loss:', score)
score = model.evaluate(vx, vy, verbose=0); print('Validation loss:', score)
score = model.evaluate(qx, qy, verbose=0); print('Test loss:', score)
#Libera todos layers do model para treinar:
for layer in model.layers: layer.trainable = True
#Treina com learning rate pequena todas as camadas
otimizador=keras.optimizers.Adam(lr=1e-6)
model.compile(otimizador, loss='categorical_crossentropy', metrics =['accuracy'])
model.fit(ax, ay, batch_size=batch_size, epochs=5, verbose=2, validation_data=(vx,vy))
score = model.evaluate(ax, ay, verbose=0); print('Training loss:', score)
score = model.evaluate(vx, vy, verbose=0); print('Validation loss:', score)
score = model.evaluate(qx, qy, verbose=0); print('Test loss:', score)
model.save(nomeprog+".h5")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Lendo arquivo 0 002a.jpg
Lendo arquivo 0 002b.jpg
Lendo arquivo 0 012a.jpg
Lendo arquivo 0 012b.jpg
Lendo arquivo 0 023a.jpg
Lendo arquivo 0 023b.jpg
Lendo arquivo 0 079a.jpg
Lendo arquivo 0 079b.jpg
Lendo arquivo 0 065a.jpg
Lendo arquivo 0 065b.jpg
Lendo arquivo 0 134a.jpg
Lendo arquivo 10 065b.jpg
Lendo arquivo 0 134b.jpg
Lendo arquivo 10 065b.jpg
Lendo arquivo 0 018a.jpg
Lendo arquivo 10 065b.jpg
Lendo arquivo 0 018b.jpg
Lendo arquivo 10 065b.jpg
Lendo arquivo 0 069a.jpg
Lendo arquivo 10 065b.jpg
Lendo arquivo 0 069b.jpg
Lendo arquivo 10 065b.jpg
Lendo arquivo 0 006a.jpg
Lendo arquivo 10 065b.jpg
Lendo arquivo 0 006b.jpg
Lendo arquivo 10 065b.jpg
Lendo arquivo 0 026a.jpg
Lendo arquivo 10 065b.jpg
Lendo arquivo 0 026b.jpg
Lendo arquivo 10 065b.jpg
Lendo arquivo 0 047a.jpg
Lendo arquivo 10 065b.jpg
Lendo arquivo 20 026b.jpg
Lendo arquivo 0 047b.jpg
Lendo arq

# Exercício 1

## Imports

In [ ]:
#resnet_transf.py - testado em Colab usando TF2
#Faz transfer learning usando ResNet50.
#No fim, faz aprendizagem descongelando todas camadas. Usando exemplo de
#https://medium.com/abraia/first-steps-with-transfer-learning-for-custom-image-classification-with-kerasb941601fcad5
import numpy as np;
import keras
import keras.backend as K;
from keras import optimizers, callbacks, regularizers;
from keras.regularizers import l2;
from keras.models import Sequential, Model;
from keras.layers import Dropout, Conv2D, MaxPooling2D, Dense, Flatten;
from keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from keras.preprocessing import image
import os; os.environ['TF_CPP_MIN_LOG_LEVEL']='3'
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Reading Files .csv

In [ ]:
def leCsv(nomeDir, nomeArq, nl=0, nc=0):
  st=os.path.join(nomeDir,nomeArq);
  arq=open(st,"rt"); lines=arq.readlines(); arq.close();
  n=len(lines)
  linhas_separadas=[]
  for linha in lines:
    linha=linha.strip('\n'); linha=linha.split(';');
    linhas_separadas.append(linha);
    ay=np.empty((n),dtype='float32');
    ax=np.empty((n,nl,nc,3),dtype='float32');
    for i in range(len(linhas_separadas)):
      if i%10==0: print("Lendo arquivo",i,linha[0])
      linha=linhas_separadas[i];
      img_path=os.path.join(nomeDir,linha[0])
      t = image.load_img(img_path, target_size=(nl,nc))
      x = image.img_to_array(t)
      x = np.expand_dims(x, axis=0)
      ax[i] = preprocess_input(x)
      ay[i] = np.float32(linha[1]); #0=m ou 1=f
  return ax, ay;
#<<<<<<<<<<<<<<<<<<<<<<<< main <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
#  Reading files
nomeprog="resnet_transf6"
#  Original: 280x200, redimensionado: 224x224
num_classes=2; nl=224; nc=224
diretorioBd="/content/drive/My Drive/Colab Notebooks/psi3472/Aula11/feiCorCrop"
ax, ay = leCsv(diretorioBd,"treino.csv", nl=nl, nc=nc); #200 imagens
qx, qy = leCsv(diretorioBd,"teste.csv", nl=nl, nc=nc); #100 imagens
vx, vy = leCsv(diretorioBd,"valida.csv", nl=nl, nc=nc); #100 imagens
ay = keras.utils.to_categorical(ay, num_classes)
qy = keras.utils.to_categorical(qy, num_classes)
vy = keras.utils.to_categorical(vy, num_classes)
input_shape = (nl,nc,3); batch_size = 10;

Lendo arquivo 0 002a.jpg
Lendo arquivo 0 002b.jpg
Lendo arquivo 0 012a.jpg
Lendo arquivo 0 012b.jpg
Lendo arquivo 0 023a.jpg
Lendo arquivo 0 023b.jpg
Lendo arquivo 0 079a.jpg
Lendo arquivo 0 079b.jpg
Lendo arquivo 0 065a.jpg
Lendo arquivo 0 065b.jpg
Lendo arquivo 0 134a.jpg
Lendo arquivo 10 065b.jpg
Lendo arquivo 0 134b.jpg
Lendo arquivo 10 065b.jpg
Lendo arquivo 0 018a.jpg
Lendo arquivo 10 065b.jpg
Lendo arquivo 0 018b.jpg
Lendo arquivo 10 065b.jpg
Lendo arquivo 0 069a.jpg
Lendo arquivo 10 065b.jpg
Lendo arquivo 0 069b.jpg
Lendo arquivo 10 065b.jpg
Lendo arquivo 0 006a.jpg
Lendo arquivo 10 065b.jpg
Lendo arquivo 0 006b.jpg
Lendo arquivo 10 065b.jpg
Lendo arquivo 0 026a.jpg
Lendo arquivo 10 065b.jpg
Lendo arquivo 0 026b.jpg
Lendo arquivo 10 065b.jpg
Lendo arquivo 0 047a.jpg
Lendo arquivo 10 065b.jpg
Lendo arquivo 20 026b.jpg
Lendo arquivo 0 047b.jpg
Lendo arquivo 10 065b.jpg
Lendo arquivo 20 026b.jpg
Lendo arquivo 0 117a.jpg
Lendo arquivo 10 065b.jpg
Lendo arquivo 20 026b.jpg
Lendo arq

## Generating .h5 files

In [ ]:
def run(loadedModel, saveName):
  print("Model ", loadedModel)
  base_model = keras.models.load_model(loadedModel)
  # print(base_model)
  base_model.layers.pop(0)
  newInput = keras.Input(shape=input_shape)    # let us say this new InputLayer
  base_model.summary()
  x = base_model(newInput)
  x = Flatten()(x)
  x = Dense(100, activation="relu")(x)
  predictions = Dense(num_classes, activation="softmax")(x)
  model = Model(inputs=newInput, outputs=predictions)
  model.summary()
  #Nao permite treinar base_model. So as camadas densas sao treinadas:
  for layer in base_model.layers: layer.trainable = False
  #Treina com learning rate grande
  otimizador=keras.optimizers.Adam(lr=1e-3)
  model.compile(otimizador, loss='categorical_crossentropy', metrics =['accuracy'])
  model.fit(ax, ay, batch_size=batch_size, epochs=5, verbose=2, validation_data=(vx,vy))
  score = model.evaluate(ax, ay, verbose=0); print('Training loss:', score)
  score = model.evaluate(vx, vy, verbose=0); print('Validation loss:', score)
  score = model.evaluate(qx, qy, verbose=0); print('Test loss:', score)
  #Libera todos layers do model para treinar:
  for layer in model.layers: layer.trainable = True
  #Treina com learning rate pequena todas as camadas
  otimizador=keras.optimizers.Adam(lr=1e-6)
  model.compile(otimizador, loss='categorical_crossentropy', metrics =['accuracy'])
  model.fit(ax, ay, batch_size=batch_size, epochs=5, verbose=2, validation_data=(vx,vy))
  score = model.evaluate(ax, ay, verbose=0); print('Training loss:', score)
  score = model.evaluate(vx, vy, verbose=0); print('Validation loss:', score)
  score = model.evaluate(qx, qy, verbose=0); print('Test loss:', score)
  model.save(saveName)

dirPath = "/content/drive/My Drive/Colab Notebooks/psi3472/Aula11/"
loadModelPath = [ dirPath + "resnet_transf0.h5",        
dirPath + "resnet_transf1.h5",
dirPath + "resnet_transf2.h5",
dirPath + "resnet_transf3.h5",
dirPath + "resnet_transf4.h5",
dirPath + "resnet_transf5.h5"] 

saveModelPath = [ dirPath + "resnet_transf_ensembled0.h5",        
dirPath + "resnet_transf_ensembled1.h5",
dirPath + "resnet_transf_ensembled2.h5",
dirPath + "resnet_transf_ensembled3.h5",
dirPath + "resnet_transf_ensembled4.h5",
dirPath + "resnet_transf_ensembled5.h5"] 

# Creating .h5 files on saveModelPath
index = 0
for loadedModel in range(0, len(loadModelPath)): 
  run(loadModelPath[index], saveModelPath[index])
  index = index + 1

Model  /content/drive/My Drive/Colab Notebooks/psi3472/Aula11/resnet_transf0.h5
Model: "functional_55"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_28 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_28[0][0]                   
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
______

## Ensamble

In [ ]:
def createEmpty(prediction):
  empty = {}
  for key, item in prediction.items():
    empty[key] = []
  return empty

def evaluatePrediction(prediction) :
  categorical  = createEmpty(prediction)
  # Convert To Categorical Male/Female 
  for key, item in prediction.items():
    for line in range(0, prediction[key].shape[0]) :
      categorical[key].append(np.argmax(prediction[key][line])) 
  tmp = []
  for key, item in categorical.items():
    tmp.append(categorical[key])
  parsed = np.stack(tmp, axis = 1)
  # Pick the most appeared
  return [ np.bincount(element).argmax() for element in parsed]

def predictions(saveModelPath, qx):
  prediction = {}
  index = 0
  # Prediction
  for modelName in saveModelPath:
    model = keras.models.load_model(modelName)
    # make predition
    prediction[index] = model.predict(qx)
  # Combine all predictions
  return evaluatePrediction(prediction)
    
# Test data
# Predict each model and pick the most appeared prediction
predicted = np.array(predictions(saveModelPath, qx))
qyTest = []
# Convert each qy line to 0 or 1
[ qyTest.append(np.argmax(test)) for test in qy] 
diff = qyTest - predicted
print(diff[diff == 0].shape)
print(predicted.shape)

(95,)
(100,)
